# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

# Configure gmaps 
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../WeatherPy/output/clean_city_data.csv')
clean_df = df.drop('Unnamed: 0', 1)
clean_df.dropna(inplace = True)
clean_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Date
0,bredasdorp,-34.5322,20.0403,80.60,54,0,10.36,1612608561
1,airai,-8.9266,125.4092,62.47,94,100,1.79,1612608558
2,sawtell,-30.3833,153.1000,73.40,94,90,4.61,1612609027
3,ishigaki,24.3448,124.1572,68.00,77,20,3.44,1612609027
4,thompson,55.7435,-97.8558,-31.00,68,40,13.80,1612608571


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations 
locations = clean_df[['Lat', 'Lng']]

In [4]:
humidity = clean_df['Humidity'].astype(float)

In [5]:
# Plot the map
fig = gmaps.figure(center=(40, 34), zoom_level=2.5)

# Create heatmap layer 
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1.5)

# Add layer 
fig.add_layer(heat_layer)

# Display map 
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
filt = (clean_df['Max temp'] > 80) | (clean_df['Max temp'] < 70) | (clean_df['Wind Speed'] > 10) | (clean_df['Cloudiness'] > 1) 
weather_condition_df = clean_df[-filt]


In [7]:
weather_condition_df['Max temp'].min()

70.0

In [8]:
weather_condition_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Date
78,hermanus,-34.4187,19.2345,79.00,60,0,3.00,1612608554
115,san rafael,-34.6177,-68.3301,73.40,48,0,6.24,1612609041
157,vao,-22.6667,167.4833,78.01,86,0,7.81,1612608559
192,vila velha,-20.3297,-40.2925,77.00,83,0,9.22,1612608630
193,rioja,-29.4111,-66.8507,73.00,53,0,3.13,1612608756
227,maldonado,-34.6667,-54.9167,70.00,88,0,7.72,1612608580
258,victoria,22.2855,114.1577,71.01,69,0,5.26,1612608421
321,keti bandar,24.1447,67.4497,77.77,23,0,8.34,1612609064
352,auriflama,-20.6856,-50.5547,70.63,77,0,5.39,1612609070
360,bam,29.1060,58.3570,77.50,14,0,4.70,1612609071


### Hotel Map
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store into variable named 'hotel_df'
hotel_df = weather_condition_df

# Add a 'Hotel Name' column to the DataFrame
hotel_df['Hotel Name'] = ""
hotel_df

/Users/aaron/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
78,hermanus,-34.4187,19.2345,79.00,60,0,3.00,1612608554,
115,san rafael,-34.6177,-68.3301,73.40,48,0,6.24,1612609041,
157,vao,-22.6667,167.4833,78.01,86,0,7.81,1612608559,
192,vila velha,-20.3297,-40.2925,77.00,83,0,9.22,1612608630,
193,rioja,-29.4111,-66.8507,73.00,53,0,3.13,1612608756,
227,maldonado,-34.6667,-54.9167,70.00,88,0,7.72,1612608580,
258,victoria,22.2855,114.1577,71.01,69,0,5.26,1612608421,
321,keti bandar,24.1447,67.4497,77.77,23,0,8.34,1612609064,
352,auriflama,-20.6856,-50.5547,70.63,77,0,5.39,1612609070,
360,bam,29.1060,58.3570,77.50,14,0,4.70,1612609071,


In [13]:
# Set parameters to search for hotels with 5000 meters.
params = {
    'radius': 5000,
    'types': 'lodging',
    'keyword': 'hotel',
    'key': g_key
}

# Loop through the DataFrame and run a lat/lng search 
for index, row in hotel_df.iterrows():
    
    # Get lat,lng from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Change location each iteration while leaveing original params in places
    params['location'] = f'{lat},{lng}'
    
    # Use search term: 'hotel' and lat/lng
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # Make request and convert to json
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
    
    except (KeyError, IndexError):
        print('Missing field/result...Skipping.')

/Users/aaron/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result...Skipping.
Missing field/result...Skipping.
Missing field/result...Skipping.


In [14]:
hotel_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
78,hermanus,-34.4187,19.2345,79.00,60,0,3.00,1612608554,Ocean Eleven
115,san rafael,-34.6177,-68.3301,73.40,48,0,6.24,1612609041,Tower Inn Hotel & Suites Casino Spa Convention...
157,vao,-22.6667,167.4833,78.01,86,0,7.81,1612608559,OURE TERA beach resort
192,vila velha,-20.3297,-40.2925,77.00,83,0,9.22,1612608630,Hotel Minuano
193,rioja,-29.4111,-66.8507,73.00,53,0,3.13,1612608756,Reina Mora Hotel
227,maldonado,-34.6667,-54.9167,70.00,88,0,7.72,1612608580,
258,victoria,22.2855,114.1577,71.01,69,0,5.26,1612608421,Rosedale Hotel Kowloon
321,keti bandar,24.1447,67.4497,77.77,23,0,8.34,1612609064,
352,auriflama,-20.6856,-50.5547,70.63,77,0,5.39,1612609070,Capoia´s Hotel
360,bam,29.1060,58.3570,77.50,14,0,4.70,1612609071,Parsian Bam Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
coordinates = hotel_df[['Lat', 'Lng']]
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))